In [1]:
# Load a triangle mesh
from lapy import TriaMesh

In [2]:
T = TriaMesh.read_off("../data/square-mesh.off")
type(T).__name__

--> OFF format         ... 
 --> DONE ( V: 415 , T: 768 )



'TriaMesh'

In [3]:
# import plotting functions
import plotly
plotly.offline.init_notebook_mode(connected=True)
from lapy.plot import plot_tria_mesh

In [4]:
# plot the mesh (here with a triangle function overlay of tria quality)
q = T.tria_qualities()
# we commented all plot calls, so that the notebook size remains small, uncomment and look at it
#plot_tria_mesh(T,plot_edges=True, tfunc=q)

In [5]:
# compute first eigenfunction
from lapy import Solver
fem = Solver(T,lump=True)
eval, evec = fem.eigs()
vfunc = evec[:,1]

# also get A,B (lumped), and inverse of B (easy as it is diagonal due to lumping)
A, B = fem.stiffness, fem.mass
Bi = B.copy()
Bi.data **= -1

TriaMesh with regular Laplace-Beltrami
Solver: spsolve (LU decomposition) ...


In [6]:
# note the mass matrix B can be used as inner product like x B x'. 
# for a lumped matrix (diagonal) it has contains the vertex areas at the diagonal. 
# in any case, the sum of all elements is the total mesh area (1 in the case of the unit square)
B.sum()

1.0

In [7]:
# when applying the Laplace to an eigenfunction you get a scaled version of that function 
#plot_tria_mesh(T,Bi*(A*vfunc),plot_edges=True)

In [8]:
# so it is the same as the corresponding eigenvalue times the eigenfunction
#plot_tria_mesh(T,eval[1]*vfunc,plot_edges=True)

In [9]:
# individually applying first gradient and then - divergence (and inverse of B) gives the same function
# inverse of B is used to get back from the integrated divergence to the function 
from lapy.diffgeo import compute_gradient
from lapy.diffgeo import compute_divergence
grad = compute_gradient(T,vfunc)
divx = -compute_divergence(T,grad)
#plot_tria_mesh(T,Bi*divx,plot_edges=True)

In [10]:
# now we compute heat diffusion from boundary vertices with default m
from lapy import heat
bvert = T.boundary_loops()
u = heat.diffusion(T,bvert,m=1)

TriaMesh with regular Laplace-Beltrami
Matrix Format now:  csc
Solver: spsolve (LU decomposition) ...


In [11]:
# show some of the level sets (not evenly spaced, steeper close to the boundary)
#plot_tria_mesh(T,u,plot_edges=True,plot_levels=True)

In [12]:
import numpy as np

# compute gradient of heat diffusion
tfunc = compute_gradient(T,u)

# normalize gradient
X = -tfunc / np.sqrt((tfunc**2).sum(1))[:,np.newaxis]
X = np.nan_to_num(X)

In [13]:
# compute divergence of normalized gradient
divx = compute_divergence(T,X)

In [14]:
# compute distance
from scipy.sparse.linalg import splu
useCholmod = True
try:
    from sksparse.cholmod import cholesky
except ImportError:
    useCholmod = False

fem = Solver(T,lump=True)
A, B = fem.stiffness, fem.mass

H=-A
b0=divx
        
# solve H x = b0
# we don't need the B matrix here, as divx is the intgrated divergence 
print("Matrix Format now: "+H.getformat())
if useCholmod:
    print("Solver: cholesky decomp - performance optimal ...")
    chol = cholesky(H)
    x = chol(b0)
else:
    print("Solver: spsolve (LU decomp) - performance not optimal ...")
    lu = splu(H)
    x = lu.solve(b0)

# remove shift
x = x-min(x)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: cholesky decomp - performance optimal ...


In [15]:
#plot_tria_mesh(T,x,plot_edges=True,plot_levels=True)

In [16]:
# max distance (smoothed)
(max(x), np.sqrt(2)/2)

(0.6049783117351546, 0.7071067811865476)

In [17]:
# we can also use the standard Poisson solver on B inverse time divx
vf = fem.poisson(-Bi*divx)
vf = vf - min(vf)
#plot_tria_mesh(T,vf,plot_edges=True,plot_levels=True)

Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


In [18]:
# should be the same as what we had earlier
max(abs(vf-x))

5.30470728232757e-07

In [19]:
# or just call this, which does all the work for us
from lapy.diffgeo import compute_geodesic_f
gf = compute_geodesic_f(T,u)
#plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


In [20]:
# again should be the same
max(abs(gf-x))

3.2569845126451114e-07

In [21]:
# testing if we can rotate the function
from lapy.diffgeo import compute_rotated_f
gf = compute_rotated_f(T,vf)
#plot_tria_mesh(T,gf,plot_edges=True,plot_levels=True)

TriaMesh with regular Laplace-Beltrami
Matrix Format now: csc
Solver: spsolve (LU decomposition) ...


In [22]:
# To demonstrate the mean curvature mapping to a sphere
# we need to have a closed mesh
from lapy.diffgeo import tria_mean_curvature_flow
from lapy.plot import plot_tria_mesh
# load your mesh here and uncomment. The mesh should have not too many flat regions (not a cube)
#T = TriaIO.import_off("../data/???")
#T2 = tria_mean_curvature_flow(T)
#plot_tria_mesh(T2,plot_edges=True,plot_levels=True)